# Capstone Project: AI Agent with PDF Knowledge Integration

## Project Overview

Congratulations on reaching the capstone project! In this project, you will build a complete **AI Agent Chatbot** that can:

1. **Learn from PDF documents** - Upload and process PDF files to expand the knowledge base
2. **Use tools intelligently** - Calculator, date/time, and knowledge search tools
3. **Maintain conversation memory** - Remember context across multiple turns
4. **Provide a user-friendly interface** - Using Streamlit

This project integrates everything you've learned in the LangChain tutorials!

---

## Learning Objectives

By completing this project, you will demonstrate proficiency in:

| Skill | Description |
|-------|-------------|
| **LCEL** | Modern chain composition with `prompt \| llm \| parser` |
| **Tool Creation** | Defining tools with `@tool` decorator |
| **LangGraph** | Building agents with `StateGraph` and explicit nodes |
| **RAG Pipeline** | Embeddings, vector stores, retrieval |
| **PDF Processing** | Document loading and text extraction |
| **Streamlit** | Building interactive web applications |
| **State Management** | Handling conversation history and session state |

---

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         AI AGENT CHATBOT ARCHITECTURE                        │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                        STREAMLIT INTERFACE                           │  │
│   │                                                                      │  │
│   │   ┌──────────────┐   ┌──────────────┐   ┌──────────────────────┐   │  │
│   │   │  PDF Upload  │   │  Chat Input  │   │   Conversation View  │   │  │
│   │   │   Widget     │   │   Widget     │   │   (History Display)  │   │  │
│   │   └──────────────┘   └──────────────┘   └──────────────────────┘   │  │
│   │                                                                      │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                    │                                        │
│                                    ▼                                        │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                    LANGGRAPH AGENT (StateGraph)                      │  │
│   │                                                                      │  │
│   │   ┌──────────────────────────────────────────────────────────────┐  │  │
│   │   │                  Agent Node (GPT-5-mini)                      │  │  │
│   │   │                                                              │  │  │
│   │   │   Decides which tools to use based on user input             │  │  │
│   │   └──────────────────────────────────────────────────────────────┘  │  │
│   │                              │                                       │  │
│   │              ┌───────────────┴───────────────┐                      │  │
│   │              │                               │                      │  │
│   │              ▼                               ▼                      │  │
│   │   ┌──────────────────────┐       ┌──────────────────────┐          │  │
│   │   │  Your Custom Tool    │       │  Knowledge Search    │          │  │
│   │   │  (You design this!)  │       │  (search_documents)  │          │  │
│   │   └──────────────────────┘       └───────────┬──────────┘          │  │
│   │                                               │                     │  │
│   └───────────────────────────────────────────────┼─────────────────────┘  │
│                                                   │                        │
│                                                   ▼                        │
│   ┌─────────────────────────────────────────────────────────────────────┐  │
│   │                     RAG PIPELINE                                     │  │
│   │                                                                      │  │
│   │   ┌──────────────┐   ┌──────────────┐   ┌──────────────────────┐   │  │
│   │   │  PDF Loader  │──>│  Embeddings  │──>│  Pinecone Vector DB  │   │  │
│   │   │  (PyPDF)     │   │  (OpenAI)    │   │  (Similarity Search) │   │  │
│   │   └──────────────┘   └──────────────┘   └──────────────────────┘   │  │
│   │                                                                      │  │
│   └─────────────────────────────────────────────────────────────────────┘  │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

LangGraph Workflow:
  START ──> agent_node ──> should_continue? ──┬──> tools_node ──> agent_node
                                              └──> END
```

---

## Requirements

### Minimum Requirements (Required for Passing)

Your project MUST include:

| Requirement | Description | Points |
|------------|-------------|--------|
| **PDF Upload** | Users can upload PDF files through the interface | 15 |
| **PDF Processing** | Extract text from PDFs and add to vector store | 15 |
| **Vector Store** | Use Pinecone to store document embeddings | 10 |
| **search_documents Tool** | Tool that retrieves relevant document chunks | 15 |
| **Custom Tool** | Create your OWN tool (be creative!) | 15 |
| **LangGraph Agent** | Build agent with StateGraph pattern | 15 |
| **Chat Interface** | Streamlit-based chat with history display | 10 |
| **Code Quality** | Clean, well-commented code | 5 |
| **TOTAL** | | **100** |

### Bonus Features (Extra Credit)

| Feature | Description | Bonus Points |
|---------|-------------|-------------|
| **Voice Input** | Add microphone input for hands-free chatting | +10 |
| **Multiple File Types** | Support for TXT, DOCX, or other formats | +10 |
| **Additional Tools** | Create more custom tools | +5 each |
| **Advanced LangGraph** | Add conditional routing or parallel nodes | +15 |
| **Deployment** | Deploy to Streamlit Cloud or similar | +10 |

---

## Implementation Guide

### Step 1: Setup and Dependencies

Create a `.env` file with your API keys:

```
OPENAI_API_KEY=your-openai-key-here
PINECONE_API_KEY=your-pinecone-key-here
```

In [ ]:
# Required imports - Add these to your Streamlit app

import os
from typing import TypedDict, Annotated
from dotenv import load_dotenv

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore

# LangGraph imports (for building agents)
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

# PDF processing
from langchain_community.document_loaders import PyPDFLoader

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# Streamlit
import streamlit as st

# Load environment variables
load_dotenv()

print("All imports successful!")

### Step 2: Create Your Tools

Tools give your chatbot special abilities! The LLM reads the tool's **docstring** to decide when to use it.

#### Required Tool
- **`search_documents`**: Searches the PDF knowledge base (this is required for the RAG feature)

#### Your Custom Tool
Create at least **ONE custom tool** of your choice. Be creative!

**Tool Ideas:**
| Tool | What It Does | Difficulty |
|------|--------------|------------|
| Word Counter | Counts words in text | Easy |
| Calculator | Does math calculations | Easy |
| Text Summarizer | Summarizes long text | Medium |
| Sentiment Analyzer | Detects positive/negative tone | Medium |
| URL Fetcher | Gets webpage content | Medium |
| Translator | Translates text to another language | Medium |

**Important**: The **docstring is crucial**! Write clear descriptions so the LLM knows WHEN to use your tool.

In [ ]:
# ============================================================
# TOOL CREATION GUIDE
# ============================================================
#
# Every tool needs 3 things:
#   1. @tool decorator
#   2. Clear docstring (LLM reads this to decide when to use it!)
#   3. Function body that returns a string
#
# TEMPLATE:
#
#   @tool
#   def my_tool_name(parameter: str) -> str:
#       """Short description of what this tool does.
#
#       Use this tool when:
#       - [Situation 1 when LLM should use this tool]
#       - [Situation 2]
#
#       Args:
#           parameter: Description of the parameter
#
#       Returns:
#           Description of what is returned
#       """
#       # Your implementation here
#       result = do_something(parameter)
#       return str(result)
#
# ============================================================


# REQUIRED TOOL: Document Search (for RAG)
# You MUST implement this tool for the PDF knowledge base feature

@tool
def search_documents(query: str) -> str:
    """Search uploaded PDF documents for relevant information.

    Use this tool when:
    - User asks about content from uploaded documents
    - User mentions 'the PDF', 'the document', or 'the file'
    - User asks questions that might be answered by document content

    Args:
        query: The search query to find relevant passages

    Returns:
        Relevant text passages from the documents, or a message if nothing found
    """
    # TODO: Implement this!
    #
    # Hint: Use something like this:
    #   docs = st.session_state.vector_store.similarity_search(query, k=3)
    #   if not docs:
    #       return "No relevant information found in the documents."
    #   results = []
    #   for i, doc in enumerate(docs, 1):
    #       results.append(f"[Passage {i}]: {doc.page_content[:500]}")
    #   return "\n\n".join(results)
    pass


# ============================================================
# YOUR CUSTOM TOOL - Be creative!
# ============================================================
#
# Pick an idea from the table above, or invent your own!
# Remember: The docstring tells the LLM WHEN to use your tool.

@tool
def your_custom_tool(input_text: str) -> str:
    """[YOUR DESCRIPTION HERE - What does this tool do?]

    Use this tool when:
    - [WHEN SHOULD THE LLM USE THIS?]
    - [ADD MORE CONDITIONS IF NEEDED]

    Args:
        input_text: [WHAT DOES THIS PARAMETER REPRESENT?]

    Returns:
        [WHAT DOES THIS TOOL RETURN?]
    """
    # TODO: Implement your creative tool here!
    #
    # Example ideas:
    # - Word counter: return f"The text has {len(input_text.split())} words."
    # - Calculator: return str(eval(input_text))  # Be careful with eval!
    # - Uppercase: return input_text.upper()
    pass

### Step 3: PDF Processing Function

Create a function to process uploaded PDF files.

In [ ]:
def process_pdf(uploaded_file):
    """
    Process an uploaded PDF file and add to vector store.

    Steps:
    1. Save the uploaded file temporarily
    2. Load the PDF using PyPDFLoader
    3. Split into chunks using RecursiveCharacterTextSplitter
    4. Add to Pinecone vector store

    Args:
        uploaded_file: Streamlit UploadedFile object

    Returns:
        str: Success or error message
    """
    # TODO: Implement PDF processing

    # Step 1: Save uploaded file temporarily
    # temp_path = f"/tmp/{uploaded_file.name}"
    # with open(temp_path, "wb") as f:
    #     f.write(uploaded_file.getbuffer())

    # Step 2: Load with PyPDFLoader (from langchain_community.document_loaders)
    # loader = PyPDFLoader(temp_path)
    # documents = loader.load()

    # Step 3: Split into chunks (from langchain_text_splitters)
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=1024,
    #     chunk_overlap=256
    # )
    # chunks = text_splitter.split_documents(documents)

    # Step 4: Add to vector store
    # st.session_state.vector_store.add_documents(chunks)

    pass

### Step 4: Create the Agent with LangGraph

Build the agent using LangGraph's `StateGraph` for explicit control over the workflow.

In [ ]:
# Define the state schema for your agent
class AgentState(TypedDict):
    """State that flows through the agent graph."""
    messages: Annotated[list, add_messages]


def create_agent_graph():
    """
    Create the agent using LangGraph StateGraph.

    Architecture:
    START -> agent_node -> should_continue? -> tools_node -> agent_node -> ...
                                            -> END

    Returns:
        Compiled LangGraph agent with memory
    """
    # Define your tools - add your custom tool here!
    tools = [search_documents, your_custom_tool]

    # Create the LLM with tool binding
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)

    # Define the agent node
    def agent_node(state: AgentState) -> dict:
        """The main agent that decides what to do."""
        system_prompt = """You are a helpful AI assistant with access to tools.

        You have tools available to help answer questions. Use them when appropriate:
        - search_documents: For finding information in uploaded PDFs
        - [Your custom tool]: [Describe when to use it]

        Always try to be helpful and use tools when they can provide better answers."""

        messages = [SystemMessage(content=system_prompt)] + state["messages"]
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}

    # Define the conditional edge function
    def should_continue(state: AgentState) -> str:
        """Decide whether to use tools or end the conversation."""
        last_message = state["messages"][-1]
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "tools"
        return "end"

    # Build the graph
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", ToolNode(tools))

    # Add edges
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {"tools": "tools", "end": END}
    )
    workflow.add_edge("tools", "agent")

    # Compile with memory for conversation persistence
    memory = MemorySaver()
    agent = workflow.compile(checkpointer=memory)

    return agent

### Step 5: Streamlit App Structure

Here's the skeleton for your Streamlit app.

In [ ]:
# Streamlit App Skeleton - Save this as 'app.py'

"""
def main():
    # Page config
    st.set_page_config(page_title="PDF Agent Chatbot", page_icon="📄")
    st.title("📄 PDF Knowledge Agent")

    # Initialize session state
    if "initialized" not in st.session_state:
        # TODO: Initialize vector store
        # TODO: Initialize LangGraph agent
        # TODO: Initialize message history
        st.session_state.agent = create_agent_graph()
        st.session_state.message_history = []
        st.session_state.thread_id = "user-session-1"
        st.session_state.initialized = True

    # Sidebar for PDF upload
    with st.sidebar:
        st.header("📤 Upload PDF")
        uploaded_file = st.file_uploader("Choose a PDF", type="pdf")

        if uploaded_file and st.button("Process PDF"):
            with st.spinner("Processing..."):
                # TODO: Call process_pdf function
                pass

    # Display chat history
    for message in st.session_state.message_history:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Chat input
    if user_input := st.chat_input("Ask a question..."):
        # Display user message
        with st.chat_message("user"):
            st.markdown(user_input)

        # Add to history
        st.session_state.message_history.append({
            "role": "user",
            "content": user_input
        })

        # Generate response using LangGraph agent
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                # Invoke the LangGraph agent
                config = {"configurable": {"thread_id": st.session_state.thread_id}}
                result = st.session_state.agent.invoke(
                    {"messages": [HumanMessage(content=user_input)]},
                    config=config
                )

                # Get the last AI message
                response = result["messages"][-1].content
                st.markdown(response)

                # Add to history
                st.session_state.message_history.append({
                    "role": "assistant",
                    "content": response
                })

if __name__ == "__main__":
    main()
"""

print("App skeleton ready! Copy this to your app.py file.")

---

## Testing Your Implementation

Use these test cases to verify your implementation works correctly:

### Test Case 1: Basic Chat
- **Input**: "Hello, how are you?"
- **Expected**: Friendly response without tool use

### Test Case 2: PDF Upload
1. Upload a sample PDF
2. Verify success message
3. Check vector store has embeddings

### Test Case 3: Document Search Tool
- **Input**: "What does the document say about [topic]?"
- **Expected**: Uses `search_documents` tool, returns relevant content from PDF

### Test Case 4: Your Custom Tool
- **Input**: [A query that should trigger YOUR custom tool]
- **Expected**: Uses your custom tool and returns appropriate result

### Test Case 5: Conversation Memory
1. Say: "My name is Alice"
2. Say: "What is my name?"
- **Expected**: Correctly remembers and returns "Alice"

---

## Submission Guidelines

### What to Submit

1. **Source Code** (Required)
   - Your complete Streamlit app (`app.py` or similar)
   - Any helper modules you created
   - `.env.example` file (WITHOUT actual API keys!)
   - `requirements.txt` with dependencies

2. **Video Demo** (Required)
   - Record a 3-5 minute video demonstrating:
     - PDF upload and processing
     - Asking questions about the PDF content
     - Using the calculator tool
     - Multi-turn conversation with memory
   - Upload to the shared folder: `luanpham.datascience@gmail.com`

3. **README** (Recommended)
   - Brief description of your implementation
   - Any bonus features you added
   - Known limitations or issues

### Evaluation Rubric

| Criteria | Excellent (90-100%) | Good (70-89%) | Needs Work (<70%) |
|----------|---------------------|---------------|-------------------|
| **Functionality** | All features work perfectly | Most features work | Major bugs |
| **Code Quality** | Clean, well-documented | Readable but minimal docs | Hard to understand |
| **Tool Integration** | Tools work seamlessly | Tools work but with issues | Tools don't work |
| **UI/UX** | Polished, intuitive | Functional but basic | Confusing or broken |
| **Demo Video** | Clear, shows all features | Shows most features | Missing key demos |

---

## Helpful Resources

### Documentation
- [LangChain Documentation](https://python.langchain.com/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Pinecone Documentation](https://docs.pinecone.io/)
- [Streamlit Documentation](https://docs.streamlit.io/)

### Sample PDFs for Testing
You can use any PDF for testing, but here are some suggestions:
- Company reports or documentation
- Academic papers
- Technical documentation
- Any text-heavy PDF

### Troubleshooting Tips

| Issue | Solution |
|-------|----------|
| API key not found | Check `.env` file and `load_dotenv()` call |
| Pinecone index error | Verify index exists and dimensions match |
| PDF not loading | Check file path and PyPDFLoader import |
| Agent not using tools | Check tool docstrings are descriptive |
| Memory not working | Verify `checkpointer` is passed to `compile()` |

---

## Good Luck!

This capstone project is your opportunity to showcase everything you've learned. Take your time, test thoroughly, and don't hesitate to revisit the tutorial notebooks if you need a refresher on any concept.

Remember:
- Start with the basic requirements first
- Test each component individually before integrating
- Use the example code in this notebook as a starting point
- Don't forget to record your demo video!

**Happy coding!** 🚀